In [1]:
import requests
from bs4 import BeautifulSoup
import re
import csv

In [2]:
csv_file = open("graphics_cards.csv", "w", newline="")
csv_writer = csv.writer(csv_file)
csv_writer.writerow(
    ["ItemID", 
    "Title", 
    "Branding", 
    "Rating", 
    "Number of Ratings", 
    "Price", 
    "Shipping", 
    "Image URL", 
    "Max Resolution", 
    "DisplayPort", 
     "HDMI", 
     "DirectX", 
     "Model"])


119

In [3]:

#         item_features = soup.find("ul", class_="item-features")
        
#         max_resolution = item_features.find('strong', string='Max Resolution:')
#         if max_resolution is not None:
#             max_resolution = max_resolution.next_sibling.strip()
#         else:
#             max_resolution = None
            
#         display_port = item_features.find('strong', string='DisplayPort:')
#         if display_port is not None:
# #             display_port = display_port.next_sibling.strip().split("x")[1].strip()
#             display_port = display_port.next_sibling.strip()
#         else:
#             display_port = None
            
#         hdmi = item_features.find('strong', string='HDMI:')
#         if hdmi is not None:
# #             hdmi = hdmi.next_sibling.strip().strip().split("x")[1].strip()
#             hdmi = hdmi.next_sibling.strip().strip()
#         else:
#             hdmi = None
            
#         directx = item_features.find('strong', string='DirectX:')
#         if directx is not None:
#             directx = directx.next_sibling.strip()
#         else:
#             directx = None
            
#         model = item_features.find('strong', string='Model #: ')
#         if model is not None:
#             model = model.next_sibling.strip()
#         else:
#             model = None
#         print(directx)

In [ ]:
import time
import random
page_limit = 10
sleep_duration = 600  # Thời gian sleep (số giây)
headers_list = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.37",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.37",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.37",
    # Thêm các User-Agent khác tùy ý vào danh sách này
]
for page in range(1, 31):
    headers = {
        "User-Agent": random.choice(headers_list)  # Chọn ngẫu nhiên một User-Agent từ danh sách
    }
    if page % page_limit == 1 & page != 1:  # Kiểm tra nếu là trang đầu tiên trong mỗi lần request
        print("sleep")
        time.sleep(sleep_duration)  # Sleep trong 5 phút
    url = f"https://www.newegg.com/GPUs-Video-Graphics-Cards/SubCategory/ID-48/Page-{page}?Tid=7709"
    print("wakeup")
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    #lấy mỗi div item
    products = soup.find_all("div", class_="item-cell")
    print(products)
    for product in products:
        item_id = product.find("div", class_="item-container").get("id")
        title = product.find("a", class_="item-title").text.strip()
        branding_element = product.find("a", class_="item-brand")
        if branding_element:
            branding = branding_element.img["title"]
        else:
            branding = None
        
        rating_element = product.find("a", class_="item-rating")
        if rating_element:
            rating = rating_element.get("title")
            rating = re.search(r"(\d+(\.\d+)?)", rating).group(1) if rating else None
        else:
            rating = None
        
        num_ratings_element = product.find("span", class_="item-rating-num")
        if num_ratings_element:
            num_ratings = re.search(r"(\d+)", num_ratings_element.text).group(1) if num_ratings_element.text else None
        else:
            num_ratings = None

        price_element = product.find("li", class_="price-current").text.strip()
        price_match = re.search(r"\$(\d{1,3}(,\d{3})*(\.\d+)?)", price_element)
        if price_match is not None:
            if price_match:
                price = float(price_match.group(1).replace(',', ''))
            else:
                price = None
        else:
            price = None
        shipping_element = product.find("li", class_="price-ship")
        if shipping_element:
            shipping = shipping_element.text.strip()
        else:
            shipping = None
            
        image_url = product.find("img")["src"]
        
        # Truy cập vào trang chi tiết sản phẩm để lấy thông tin chi tiết
        details_url = product.find("a", class_="item-img")["href"]
        details_response = requests.get(details_url)
        details_soup = BeautifulSoup(details_response.text, "html.parser")
        
        
        # Tìm tất cả các bảng có class "table-horizontal"
        tables = details_soup.find_all("table", class_="table-horizontal")

        # Tạo một từ điển để lưu trữ thông tin
        details = {}

        # Lặp qua từng bảng để lấy thông tin
        for table in tables:
            caption = table.caption.get_text(strip=True)
            if caption in ["Model", "3D API", "Ports", "Details"]:
                rows = table.tbody.find_all("tr")
                for row in rows:
                    th = row.th.get_text(strip=True)
                    td = row.td.get_text(strip=True)
                    if th in ["Max Resolution", "DisplayPort", "HDMI", "DirectX", "Model"]:
                        details[th] = td

        max_resolution = details.get("Max Resolution", "")
        display_port = details.get("DisplayPort", "")
        hdmi = details.get("HDMI", "")
        directx = details.get("DirectX", "")
        model = details.get("Model", "")
        csv_writer.writerow([item_id, title, branding, rating, num_ratings, price, shipping, image_url, max_resolution, display_port, hdmi, directx, model])
    print(f"Page {page} done.")
print("csv_writer done")
csv_file.close()

wakeup
[<div class="item-cell" id="item_cell_14-932-580_1_0"><div class="item-container" id="14-932-580"><a class="item-img" href="https://www.newegg.com/gigabyte-geforce-rtx-4070-ti-gv-n407tgaming-oc-12gd/p/N82E16814932580"><img alt="GIGABYTE GeForce RTX 4070 Ti GAMING OC 12G Graphics Card, 3x WINDFORCE Fans, 12GB 192-bit GDDR6X, GV-N407TGAMING OC-12GD Video Card" src="https://c1.neweggimages.com/ProductImageCompressAll300/14-932-580-15.jpg" title="GIGABYTE GeForce RTX 4070 Ti GAMING OC 12G Graphics Card, 3x WINDFORCE Fans, 12GB 192-bit GDDR6X, GV-N407TGAMING OC-12GD Video Card"/><div class="btn btn-large btn-quickview">Quick View</div></a><div class="item-info"><div class="item-branding"><a class="item-brand" href="https://www.newegg.com/GIGABYTE/BrandStore/ID-1314"><img alt="GIGABYTE" src="https://c1.neweggimages.com/Brandimage_70x28/Brand1314.gif" title="GIGABYTE"/></a><a class="item-rating" href="https://www.newegg.com/gigabyte-geforce-rtx-4070-ti-gv-n407tgaming-oc-12gd/p/N82E1681

Page 1 done.
wakeup
[<div class="item-cell" id="item_cell_14-500-538_1_0"><div class="item-container" id="14-500-538"><a class="item-img" href="https://www.newegg.com/zotac-geforce-rtx-4090-zt-d40900b-10p/p/N82E16814500538"><img alt="ZOTAC GAMING GeForce RTX 4090 AMP Extreme AIRO DLSS 3 24GB GDDR6X 384-bit 21 Gbps PCIE 4.0 Gaming Graphics Card, IceStorm 3.0 Advanced Cooling, SPECTRA 2.0 RGB Lighting, ZT-D40900B-10P" src="https://c1.neweggimages.com/ProductImageCompressAll300/14-500-538-16.jpg" title="ZOTAC GAMING GeForce RTX 4090 AMP Extreme AIRO DLSS 3 24GB GDDR6X 384-bit 21 Gbps PCIE 4.0 Gaming Graphics Card, IceStorm 3.0 Advanced Cooling, SPECTRA 2.0 RGB Lighting, ZT-D40900B-10P"/><div class="btn btn-large btn-quickview">Quick View</div></a><div class="item-info"><div class="item-msg is-best-price"><div class="flags flags-right flags-red is-invert"><a class="flags-body" href="https://www.newegg.com/d/Lowest-Price-In-30-Days/GPUs-Video-Graphics-Cards/s/ID-48" title="GPUs / Video Grap

Page 2 done.
wakeup
[]
Page 3 done.
wakeup
[]
Page 4 done.
wakeup
[]
Page 5 done.
wakeup
[]
Page 6 done.
wakeup
[]
Page 7 done.
wakeup
[]
Page 8 done.
wakeup
[]
Page 9 done.
sleep
